#### How to choose the granularity of the context such that the agent can choose spans

In [2]:
import nltk
import spacy
import benepar

from datasets import load_dataset

benepar.download('benepar_en3')
# squad = load_dataset("readerbench/squad_qgen", split="train")

[nltk_data] Downloading package benepar_en3 to /Users/teo/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


True

In [3]:
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

In [7]:
psy = open("psy.txt", "r").read()

doc = nlp(psy)

for sent in doc.sents:
    const_list = list(sent._.constituents)
    for const in const_list:
        print(const)

Industrial-Organizational psychology (I-O psychology) is a subfield of psychology that applies psychological
theories, principles, and research findings in industrial and organizational settings.
Industrial-Organizational psychology (I-O psychology)
Industrial-Organizational psychology
Industrial-Organizational
Industrial
-
Organizational
psychology
(
I-O psychology
I-O
I
-
O
psychology
)
is a subfield of psychology that applies psychological
theories, principles, and research findings in industrial and organizational settings
is
a subfield of psychology that applies psychological
theories, principles, and research findings in industrial and organizational settings
a subfield of psychology
a subfield
a
subfield
of psychology
of
psychology
that applies psychological
theories, principles, and research findings in industrial and organizational settings
that
applies psychological
theories, principles, and research findings in industrial and organizational settings
applies
psychological
the

In [4]:
# generate a dataframe with sentence and all possible answers
import pandas as pd
squad_df = []

from tqdm import tqdm
# iterate through the dataset
for i, sample in tqdm(enumerate(squad), total=len(squad)):

    # eliminate the contexts that have sentences longer than 512 tokens
    try:
        doc = nlp(sample['context'])
    except:
        continue

    for sent in doc.sents:

        sent_text = sent.text

        constit_list = list(sent._.constituents)

        tag_constit_list = [(constit._.labels, constit.text.strip()) for constit in constit_list]

        sample['possible_answers'] = tag_constit_list

        squad_df.append((sent_text, tag_constit_list))

  0%|          | 0/82900 [00:00<?, ?it/s]/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
100%|██████████| 82900/82900 [24:03:06<00:00,  1.04s/it]      


In [5]:
squad_df_new = pd.DataFrame(squad_df, columns=['sentence', 'spans'])
squad_df_new.head()


,sentence,spans
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[((S,), Beyoncé Giselle Knowles-Carter (/biːˈj..."
1,"Born and raised in Houston, Texas, she perform...","[((S,), Born and raised in Houston, Texas, she..."
2,"Managed by her father, Mathew Knowles, the gro...","[((S,), Managed by her father, Mathew Knowles,..."
3,Their hiatus saw the release of Beyoncé's debu...,"[((S,), Their hiatus saw the release of Beyonc..."
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[((S,), Beyoncé Giselle Knowles-Carter (/biːˈj..."


In [ ]:
squad_df_new.to_json('squad_df_train.json')